In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Specify HyperParameters

In [3]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.2"
device = "cuda:0"
dataset_name = "../workdir/data/triviaqa.csv"
batch_size = 4
seed = 42

# Initialize Model

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    load_in_8bit=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

generation_config = GenerationConfig.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
messages = [
    [
        {
            "role": "user", 
            "content": "How many fingers on a coala's foot?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Who sang a song Yesterday?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Кто спел песню Кукла Колдуна?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Translate into French: 'I want a small cup of coffee'"
        }
    ]
]

chat_messages = [tokenizer.apply_chat_template(m, tokenize=False) for m in messages]
tokenizer.pad_token = tokenizer.eos_token
encoded = tokenizer(chat_messages, padding=True, return_tensors="pt")

In [6]:
from lm_polygraph.stat_calculators.basic_probs import BasicGreedyProbsCalculatorCausalLM
from lm_polygraph.stat_calculators.entropy import EntropyCalculator
from lm_polygraph.estimators import MeanTokenEntropy

from torch.utils.data import DataLoader


model_wrapper = BasicGreedyProbsCalculatorCausalLM()
calc_entropy = EntropyCalculator()

args_generate = {"generation_config" : generation_config,
                 "max_new_tokens": 30}

#estimator = MaximumSequenceProbability()
estimator = MeanTokenEntropy()

data_loader = DataLoader(encoded, batch_size=2, shuffle=False, collate_fn=lambda x: x)

for batch in data_loader:
    out = model_wrapper(encoded, model, args_generate)
    out.update(calc_entropy(out))

    uncertianty_score = estimator(out)
    result = tokenizer.batch_decode(out['greedy_tokens'])

    for text, ue_score in zip(result, uncertianty_score):
        print("Output:", text)
        print("UE score:", ue_score)
        print()

/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
2024-04-07 22:45:41.5

Output: A koala does not have webbed feet like a coala from the comic books or cartoons. A koala's feet are actually
UE score: 1.364561e-05

Output: The song "Yesterday" was written and first performed by the English singer-songwriter Paul McCartney. It was released as a single
UE score: 1.0055466e-05

Output: The song "Kukla Koldun" is a popular Russian children's song. The original version was recorded by the Soviet children's
UE score: 2.0145751e-05

Output: "Je veux une tasse petite de café" is the correct translation in French for "I want a small cup of coffee".</s>
UE score: 7.944358e-06

